## Цель этапа обучения

Цель данного этапа — обучить базовую LSTM-модель
для бинарной классификации отзывов на положительные и отрицательные.

Данная модель используется как baseline (точка отсчёта),
с которой в дальнейшем будут сравниваться улучшенные архитектуры.


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, Dataset
import pickle
from pathlib import Path

## Загрузка данных и словаря


In [2]:
ARTIFACTS_DIR = Path("artifacts")
data = torch.load(ARTIFACTS_DIR / "dataset.pt")
X_train = data["X_train"]
y_train = data["y_train"]
X_val = data["X_val"]
y_val = data["y_val"]
X_test = data["X_test"]
y_test = data["y_test"]
with open(ARTIFACTS_DIR / "word2idx.pkl", "rb") as f:
    word2idx = pickle.load(f)
VOCAB_SIZE = len(word2idx)
X_train.shape, y_train.shape

C:\Users\nurs\AppData\Local\Temp\ipykernel_1864\1425761307.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(ARTIFACTS_DIR / "dataset.pt")


(torch.Size([35000, 400]), torch.Size([35000]))

На данном этапе используется уже подготовленный датасет,
сохранённый на этапе Data Preprocessing.
Повторная обработка текста не выполняется.


## Подготовка Dataset и DataLoader


In [4]:
class IMDBTensorDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return self.X.size(0)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [5]:
BATCH_SIZE = 64
train_dataset = IMDBTensorDataset(X_train, y_train)
val_dataset   = IMDBTensorDataset(X_val, y_val)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [6]:
batch_X, batch_y = next(iter(train_loader))
batch_X.shape, batch_y.shape

(torch.Size([64, 400]), torch.Size([64]))

DataLoader обеспечивает:
- обучение батчами
- перемешивание данных на этапе обучения
- корректную подачу данных в модель


## Архитектура базовой LSTM-модели


In [7]:
class LSTMClassifier(nn.Module):
    def __init__(
        self,
        vocab_size,
        embedding_dim,
        hidden_dim,
        padding_idx
    ):
        super().__init__()

        # 1) Embedding слой
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_dim,
            padding_idx=padding_idx
        )
        # 2) LSTM слой
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            batch_first=True
        )
        # 3) Выходной слой
        self.fc = nn.Linear(hidden_dim, 1)
    def forward(self, x):
        """
        x: (batch_size, seq_len)
        """
        # (batch, seq) → (batch, seq, embed)
        embedded = self.embedding(x)
        # LSTM
        # hidden: (num_layers, batch, hidden_dim)
        _, (hidden, _) = self.lstm(embedded)
        # берём последнее скрытое состояние
        hidden = hidden.squeeze(0)
        # (batch, hidden_dim) → (batch, 1)
        logits = self.fc(hidden)
        return logits.squeeze(1)

In [8]:
PAD_IDX = 0
EMBEDDING_DIM = 128
HIDDEN_DIM = 128
model = LSTMClassifier(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    padding_idx=PAD_IDX
)
model

LSTMClassifier(
  (embedding): Embedding(20000, 128, padding_idx=0)
  (lstm): LSTM(128, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
)

Модель состоит из:
- embedding слоя
- одного LSTM слоя
- линейного классификатора

Выход модели — логиты, используемые для бинарной классификации.


In [11]:
with torch.no_grad():
    logits = model(batch_X)
logits.shape

torch.Size([64])

## Функция потерь и оптимизатор


In [12]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.train()
batch_X, batch_y = next(iter(train_loader))
batch_X = batch_X.to(device)
batch_y = batch_y.to(device)
optimizer.zero_grad()
logits = model(batch_X)
loss = criterion(logits, batch_y)
loss.backward()
optimizer.step()
loss.item()

0.691602885723114

Используется функция потерь BCEWithLogitsLoss,
которая является стандартом для бинарной классификации
и численно более стабильна, чем sigmoid + BCELoss.


## Цикл обучения


In [13]:
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0

    for X_batch, y_batch in dataloader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)


In [14]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            total_loss += loss.item()

    return total_loss / len(dataloader)


In [15]:
EPOCHS = 5
for epoch in range(1, EPOCHS + 1):
    train_loss = train_one_epoch(
        model, train_loader, optimizer, criterion, device
    )
    val_loss = evaluate(
        model, val_loader, criterion, device
    )

    print(
        f"Epoch {epoch:02d} | "
        f"Train Loss: {train_loss:.4f} | "
        f"Val Loss: {val_loss:.4f}"
    )

Epoch 01 | Train Loss: 0.6930 | Val Loss: 0.6927
Epoch 02 | Train Loss: 0.6844 | Val Loss: 0.6932
Epoch 03 | Train Loss: 0.6663 | Val Loss: 0.6782
Epoch 04 | Train Loss: 0.6560 | Val Loss: 0.7050
Epoch 05 | Train Loss: 0.6431 | Val Loss: 0.7117


В процессе обучения наблюдается снижение training loss,
однако validation loss начинает расти после нескольких эпох,
что указывает на переобучение базовой модели.


## Оценка качества модели


In [ ]:
def compute_accuracy(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            logits = model(X_batch)
            probs = torch.sigmoid(logits)
            preds = (probs >= 0.5).float()
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)
    return correct / total


In [17]:
val_accuracy = compute_accuracy(model, val_loader, device)
print(f"Validation Accuracy: {val_accuracy:.4f}")

Validation Accuracy: 0.5045


In [ ]:
test_dataset = IMDBTensorDataset(X_test, y_test)
test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False
)
test_accuracy = compute_accuracy(model, test_loader, device)
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.5068


Accuracy используется как основная метрика,
поскольку датасет IMDb является сбалансированным.


## Сохранение базовой модели


In [19]:
MODEL_DIR = Path("models")
MODEL_DIR.mkdir(exist_ok=True)
torch.save(
    model.state_dict(),
    MODEL_DIR / "baseline_lstm.pt"
)

## Итог

Была обучена базовая LSTM-модель для задачи анализа тональности отзывов IMDb.

Модель демонстрирует обучение на тренировочных данных,
однако её качество на validation и test выборках
близко к случайному, что указывает на ограниченность архитектуры.

Данная модель используется как baseline.
Следующим этапом является улучшение архитектуры
с использованием BiLSTM.
